In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
dataset=pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
dataset.head()

2024-05-23 12:08:04.792117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 12:08:04.792224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 12:08:04.926233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [2]:
len(dataset['en'][1])

8

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer))
def embedder(text):
    global tokenizer,model
    input_ids = tokenizer(text, return_tensors='tf')
    arr=text.split(" ")
    num_added_toks = tokenizer.add_tokens(arr)
    model.resize_token_embeddings(len(tokenizer))
    outputs = model(input_ids)
    #Applying Padding
    if outputs[0].shape[1]>100:
        cls_embedding=outputs[0][:,:100,:]
    else:
        cls_embedding=tf.concat([outputs[0], tf.zeros([1,100-outputs[0].shape[1],768])], axis=1)
    #Positional encoding 
    seq_len,d,n=100,768,10000
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    #Adding positional encoding
    cls_embedding += [P]
    return cls_embedding
    

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[ BEFORE ] tokenizer vocab size: 105879


In [4]:
embedder("Some weights of the PyTorch model")

<tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
array([[[-0.14002718,  0.99718755, -0.02695617, ...,  0.97366893,
          0.05828621,  0.9977569 ],
        [ 0.6059175 ,  0.51176375,  0.79083943, ...,  0.8594985 ,
          0.66855276,  0.7021471 ],
        [ 0.42150906, -0.32307467,  1.172154  , ...,  0.7035326 ,
          0.40775174,  0.43801546],
        ...,
        [ 0.37960774, -0.92514753,  0.4379616 , ...,  0.9999482 ,
          0.00993531,  0.99995065],
        [-0.5733819 , -0.81928825,  0.9900526 , ...,  0.99994713,
          0.01003773,  0.99994963],
        [-0.99920684,  0.03982088,  0.67107844, ...,  0.99994606,
          0.01014015,  0.99994856]]], dtype=float32)>

In [5]:
import math
def encoder(embed):
    # weights of query ,keys and values and other weights
    Wq=tf.random.normal(shape=(1,100, 768))
    Wk=tf.random.normal(shape=(1,100, 768))
    Wv=tf.random.normal(shape=(1,100, 768))
    W0=tf.random.normal(shape=(1,100, 768))
    W1=tf.random.normal(shape=(1,100, 768))
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    Key=normalized_embeddings*Wk
    Value=normalized_embeddings*Wv
    return [Key,Value]

In [6]:
encoder(embedder("Some weights of the PyTorch model"))

[<tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
 array([[[-0.03997879,  0.99544185,  0.40684092, ..., -1.2193112 ,
          -0.17893527,  0.1885353 ],
         [ 0.13836475, -0.01264598,  0.49097738, ...,  0.68124026,
           1.9806463 , -0.6978776 ],
         [-0.23685463,  0.02821267, -0.8088831 , ..., -1.3365338 ,
           0.68238336,  0.00437639],
         ...,
         [-0.12918742, -3.9966738 , -0.02609568, ...,  0.15455694,
          -0.01030072, -0.83525157],
         [ 0.72374505, -0.02801639,  0.04396672, ...,  0.18179208,
           0.13703552,  0.06563742],
         [-0.32466757, -0.02942987, -0.46013126, ...,  0.10932817,
           0.3654009 , -1.1851817 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 100, 768), dtype=float32, numpy=
 array([[[-1.2074107 , -0.2947543 ,  0.2398512 , ...,  0.6971482 ,
          -0.45831376, -1.0767586 ],
         [-0.162456  , -0.08075656, -0.10560892, ...,  0.1010467 ,
           0.43407542,  0.73629653],
         [-0.08947724, 

In [7]:
def decoder(embed, arr):
    # weights of query ,keys and values and other weights
    Wq=tf.random.normal(shape=(1,100, 768))
    Wk=tf.random.normal(shape=(1,100, 768))
    Wv=tf.random.normal(shape=(1,100 ,768))
    W0=tf.random.normal(shape=(1,100 ,768))
    W1=tf.random.normal(shape=(1,100 ,768))
    W01=tf.random.normal(shape=(1,100, 768))
    W11=tf.random.normal(shape=(1,100, 768))
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    embed = tf.keras.layers.LayerNormalization()(embed)
    #key ,value of encodings output
    Key,Value=arr
    Query=embed*Wq
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W01*normalized_embeddings
    result1=tf.nn.relu(Layer1)
    result=W11*normalized_embeddings
    #Add layer
    embed+=result
    return embed

In [8]:
print(decoder(embedder(dataset['fr'][1]),encoder(embedder(dataset['en'][1]))))

tf.Tensor(
[[[-6.7368555e-01 -4.1124761e-01  9.8122013e-01 ...  7.8307372e-01
   -1.1798513e+00  5.5705383e-04]
  [ 4.1951868e-01  2.1674709e+00  3.1219921e-01 ...  6.9475138e-01
   -5.5850327e-01 -4.0458232e-02]
  [-2.2127301e-01 -5.4418402e+00 -4.5244277e-02 ...  1.9208432e+00
   -8.6443320e-02 -1.5811067e-02]
  ...
  [ 9.7701210e-04  2.3389513e+00  4.4990498e-01 ...  3.9247498e-02
   -1.8579082e-01  4.8873794e-01]
  [-1.1088663e+00 -1.7259994e+00 -1.9963450e+00 ... -1.9987239e-01
   -1.9741693e-01  1.6140263e+00]
  [-1.8208870e+00 -1.5917355e+00  4.7765470e-01 ...  4.3163824e-01
   -5.2318585e-01 -5.5253364e-02]]], shape=(1, 100, 768), dtype=float32)


In [9]:
def train(input1,output1):
    global model,tokenizer
    W0=tf.random.normal(shape=(1,100, 768))
    encoder_output=encoder(embedder(input1))
    output_embed=embedder(output1)
    decoder_output=decoder(output_embed,encoder_output)
    Layer1=W0*decoder_output
    result=tf.nn.softmax(Layer1, axis=-1)
    predicted_token_ids = tf.argmax(result, axis=-1).numpy().flatten()
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids)
    predicted_text = tokenizer.convert_tokens_to_string(predicted_tokens)
    
    input_ids = tokenizer(output1, return_tensors='tf')
    outputs = model(input_ids)
    if outputs[0].shape[1]>100:
        cls_embedding=outputs[0][:,:100,:]
    else:
        cls_embedding=tf.concat([outputs[0], tf.zeros([1,100-outputs[0].shape[1],768])], axis=1)
    print(cls_embedding.shape,result.shape)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=cls_embedding, logits=result))
    total_loss = cross_entropy 
#     gradients = tf.gradients(loss, model.trainable_variables)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    print(total_loss)
    return predicted_text

In [10]:
print(train(dataset['fr'][1],dataset['en'][1]))

(1, 100, 768) (1, 100, 768)
tf.Tensor(0.40218288, shape=(), dtype=float32)
ફ } ৱ [unused90] ओ [unused29] [unused47] [unused49] [unused54] [unused9] ઓ [unused17] [unused94] a [unused86] : 8 ־ } <S> و ® ɨ ? q [unused89] ¼ [unused12] [unused14] ʷ d ־ [unused6] [unused37] [unused94] w [unused94] ভ . [unused77] [unused15] [unused10] ɐ ð ɦ ± [unused78] \ स [unused64] ط [unused14] झ ʁ گ ط ¿ ھ [unused14] ǁ [unused63] ( ॐ [unused37] २ צ ʳ [unused38] ʕ [unused95] ɹ ւ £ [unused1] ɯ ɓ ĳ ʒ ʉ લ [unused57] ھ ʧ ʦ ^ ॉ ъ « [unused98] ɔ ɹ ¦ ट [unused34] ٫ [unused14] گ ʎ ʀ [unused84]
